
# Binary‑Breathing Deep‑Dive (v3)

* **Streaming histogram** keeps memory < 2 GB for N ≤ 30.  
* Skip‑first‑bin logic so peak finder ignores the trivially over‑populated 0–1e‑7 range.  
* Parameters cell lets you toggle power = 2 / 3, bit inversion, precision bump.  
* Outputs peak list, gap ratios, log plot, and minimal seed search (≤ 40 bits).

Run every cell top‑to‑bottom in Colab.  Tweak the parameter cell only.


In [ ]:

import numpy as np, matplotlib.pyplot as plt, time, gc
from mpmath import mp
mp.dps = 50


In [ ]:

# ===== PARAMETERS =====
MAX_LEN   = 30        # deepest bit-length
BIN_WIDTH = 1e-7      # histogram resolution
POWER     = 2         # 2=square, 3=cube
INVERT    = False     # reverse bit order
CUT       = 1e-4      # only keep y <= CUT
SEED_TOL  = 1e-12     # tolerance for seed search
# ======================


In [ ]:

def bits_to_value(i, n, invert=False):
    b = format(i, f'0{n}b')
    if invert:
        b = b[::-1]
    return int(b, 2) / (1 << n)

def streaming_hist_n(n):
    bins = np.arange(0, CUT + BIN_WIDTH, BIN_WIDTH)
    counts = np.zeros(len(bins)-1, dtype=np.int32)
    for i in range(1, 1 << n):
        y = bits_to_value(i, n, INVERT) ** POWER
        if y <= CUT:
            counts[int(y // BIN_WIDTH)] += 1
    return bins[:-1], counts


In [4]:

peak_positions = []
for n in range(20, MAX_LEN+1):
    xs, cs = streaming_hist_n(n)
    idx = cs[1:].argmax() + 1   # skip first bin
    peak_x = xs[idx] + 0.5*BIN_WIDTH
    peak_positions.append((n, peak_x))
    print(f"N={n:2d}  peak={peak_x:.9e}")


N=20  peak=1.500000000e-07
N=21  peak=1.500000000e-07
N=22  peak=1.500000000e-07
N=23  peak=1.500000000e-07
N=24  peak=1.500000000e-07
N=25  peak=1.500000000e-07
N=26  peak=1.500000000e-07
N=27  peak=1.500000000e-07
N=28  peak=1.500000000e-07
N=29  peak=1.500000000e-07


KeyboardInterrupt: 

In [ ]:

gaps = [peak_positions[i-1][1] - peak_positions[i][1] for i in range(1, len(peak_positions))]
ratios = [gaps[i-1] / gaps[i] for i in range(1, len(gaps))]

plt.figure(figsize=(5,3))
plt.plot([n for n,_ in peak_positions], [x for _,x in peak_positions], 'o-')
plt.yscale('log')
plt.xlabel('Bit length N'); plt.ylabel('Peak x')
plt.title('Peak lattice (log‑y)')
plt.show()

if ratios:
    plt.figure(figsize=(4,3))
    plt.plot(range(21, 21+len(ratios)), ratios, 'o-')
    plt.axhline(4.669, color='r', ls='--', label='Feigenbaum δ')
    plt.xlabel('N'); plt.ylabel('Gap ratio Δ_{n-1}/Δ_n')
    plt.legend(); plt.title('Gap‑ratio convergence')
    plt.show()


In [ ]:

def find_seed(target, tol=SEED_TOL, max_len=40):
    for n in range(1, max_len+1):
        denom = 1 << n
        for i in range(1, denom):
            x = i / denom
            if abs(x**POWER - target) < tol:
                return n, format(i, f'0{n}b')
    return None, None

print("\nMinimal seeds:")
for n, px in peak_positions:
    slen, sbits = find_seed(px)
    if slen:
        print(f"N={n}  seed length={slen} bits  prefix={sbits[:16]}...")
    else:
        print(f"N={n}  (no seed found ≤40 bits)")
